In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from datetime import datetime
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
online_action = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', engine='python', encoding='utf-8-sig')
transaction = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', engine='python', encoding='utf-8-sig')
customer = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv', engine='python', encoding='utf-8-sig')
production = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv', engine='python', encoding='utf-8-sig')

In [6]:
transaction2 = pd.read_csv('clac_nm.csv', engine='python', encoding='utf-8-sig')

In [8]:
transaction2['trans_tm']=transaction2.de_dt.astype(str) + ' ' + transaction2.de_tm

In [9]:
transaction2.trans_tm=transaction2.trans_tm.apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:8] + x[8:])

In [10]:
import datetime
transaction2.trans_tm = transaction2.trans_tm.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M'))

In [11]:
transaction2['trans_tm_month']=transaction2.trans_tm.apply(lambda x: x.month)

In [133]:
# 이상치 제거
transaction2.drop(354563, inplace =True)

In [15]:
customer.clnt_age.value_counts()

unknown    60154
40          4948
30          4307
50          1694
20           924
60           326
10            46
Name: clnt_age, dtype: int64

In [16]:
customer.drop(customer[customer['clnt_age'] == 'unknown'].index, inplace = True)

In [17]:
transaction3 = pd.merge(transaction2, customer, on = 'clnt_id', how = 'left')

In [18]:
transaction3.drop('Unnamed: 0', axis=1,inplace=True)

In [19]:
transaction3.clnt_age.value_counts()

40    268827
30    189791
50     90025
20     19412
60     13987
10       267
Name: clnt_age, dtype: int64

In [20]:
transaction_ages_payment=transaction3.pivot_table(index=['clnt_age', 'clac_nm0', 'trans_tm_month'], values='buy_am', aggfunc='sum')

In [21]:
transaction_ages_count=transaction3.pivot_table(index=['clnt_age', 'clac_nm0', 'trans_tm_month'], values='buy_am', aggfunc='count')

In [146]:
transaction_ages_count.columns=['clnt_num']

In [23]:
# 평균 내기 But, 7,8,9월 합이 500 이하인 경우는 Trend를 반영하기 힘들다고 간주하여 전국민 대상 쇼핑 동향으로 대체
trend_korea = pd.read_csv('온오프라인 쇼핑 동향.csv', engine='python', encoding='utf-8-sig')

In [62]:
trend_korea.columns=['clac_nm0', 'trans_tm_month', 'buy_am_korea', 'trend_korea']

In [153]:
trend_korea.set_index(['clac_nm0', 'trans_tm_month'])

buy_am_korea  trend_korea
clac_nm0            trans_tm_month                          
Baby                7                    39,659     0.067508
                    8                    41,710     0.051716
                    9                    41,681    -0.000695
Beauty              7                   696,542     0.050069
                    8                   740,689     0.063380
                    9                   803,690     0.085057
Digiter, Appliences 7                   380,139     0.057181
                    8                   341,060    -0.102802
                    9                   300,333    -0.119413
Fashion_Accessories 7                   226,420     0.052881
                    8                   209,674    -0.073960
                    9                   240,026     0.144758
Fashion_Clothes     7                   380,096     0.041898
                    8                   311,563    -0.180304
                    9                   454,911     0.460093
Foods               7                   459,618     0.074095
                    8                   517,049     0.124954
                    9                   519,648     0.005027
Interier            7                    68,424    -0.011171
                    8                    61,196    -0.105635
                    9                    64,340     0.051376
Living, Health      7                   302,097     0.140639
                    8                   308,283     0.020477
                    9                   305,203    -0.009991
Others              7                    52,205     0.118120
                    8                    51,134    -0.020515
                    9                    50,228    -0.017718
Sports, Leisure     7                    97,407    -0.031229
                    8                   113,798     0.168273
                    9                    72,839    -0.359927
Travel, Culture     7                   813,189     0.074720
                    8                   806,915    -0.007715
                    9                   713,819    -0.115373

In [25]:
transaction_ages_payment['trend_lpoint'] = 0

In [51]:
for j in ['20', '30', '40', '50', '60']:
    for i in range(0, len(transaction_ages_payment.loc[j])):
        if (i % 3 == 1) | (i % 3 == 2):
            transaction_ages_payment.loc[j].iloc[i][1] = \
            100*(transaction_ages_payment.loc[j].iloc[i][0]-transaction_ages_payment.loc[j].iloc[i-1][0])/transaction_ages_payment.loc['20'].iloc[i-1][0]
            transaction_ages_payment.loc[j].iloc[i][1]=transaction_ages_payment.loc[j].iloc[i][1].astype(float)

In [64]:
transaction_10_payment=transaction_ages_payment.loc['10'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])
transaction_20_payment=transaction_ages_payment.loc['20'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])
transaction_30_payment=transaction_ages_payment.loc['30'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])
transaction_40_payment=transaction_ages_payment.loc['40'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])
transaction_50_payment=transaction_ages_payment.loc['50'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])
transaction_60_payment=transaction_ages_payment.loc['60'].reset_index(['clac_nm0', 'trans_tm_month']).merge(trend_korea,on=['clac_nm0', 'trans_tm_month'])

In [66]:
# 10대의 경우 구매인원이 너무 적어서 제외 -> korea trend랑 동일한 것으로 생각
transaction_10_payment.trend_lpoint=transaction_10_payment.trend_korea
transaction_20_payment.trend_lpoint=transaction_20_payment.trend_lpoint/100
transaction_30_payment.trend_lpoint=transaction_30_payment.trend_lpoint/100
transaction_40_payment.trend_lpoint=transaction_40_payment.trend_lpoint/100
transaction_50_payment.trend_lpoint=transaction_50_payment.trend_lpoint/100
transaction_60_payment.trend_lpoint=transaction_60_payment.trend_lpoint/100

In [119]:
for i in range(0,32):
    if transaction_20_payment.trend_lpoint[i] == 0.0:
        transaction_20_payment.trend_lpoint[i] = transaction_20_payment.trend_korea[i]
for i in range(0,32):
    if transaction_30_payment.trend_lpoint[i] == 0.0:
        transaction_30_payment.trend_lpoint[i] = transaction_30_payment.trend_korea[i]
for i in range(0,32):
    if transaction_40_payment.trend_lpoint[i] == 0.0:
        transaction_40_payment.trend_lpoint[i] = transaction_40_payment.trend_korea[i]
for i in range(0,32):
    if transaction_50_payment.trend_lpoint[i] == 0.0:
        transaction_50_payment.trend_lpoint[i] = transaction_50_payment.trend_korea[i]
for i in range(0,32):
    if transaction_60_payment.trend_lpoint[i] == 0.0:
        transaction_60_payment.trend_lpoint[i] = transaction_60_payment.trend_korea[i]

In [121]:
transaction_10_payment['trend_mean'] = (transaction_10_payment.trend_lpoint + transaction_10_payment.trend_korea) / 2
transaction_20_payment['trend_mean'] = (transaction_20_payment.trend_lpoint + transaction_20_payment.trend_korea) / 2
transaction_30_payment['trend_mean'] = (transaction_30_payment.trend_lpoint + transaction_30_payment.trend_korea) / 2
transaction_40_payment['trend_mean'] = (transaction_40_payment.trend_lpoint + transaction_40_payment.trend_korea) / 2
transaction_50_payment['trend_mean'] = (transaction_50_payment.trend_lpoint + transaction_50_payment.trend_korea) / 2
transaction_60_payment['trend_mean'] = (transaction_60_payment.trend_lpoint + transaction_60_payment.trend_korea) / 2

In [155]:
transaction_20_payment.set_index(['clac_nm0', 'trans_tm_month'])

buy_am  trend_lpoint buy_am_korea  \
clac_nm0            trans_tm_month                                        
Baby                7                 569650      0.067508       39,659   
                    8                 868440      0.520000       41,710   
                    9                 718410     -0.170000       41,681   
Beauty              7                4889170      0.050069      696,542   
                    8                4757470     -0.020000      740,689   
                    9                4117675     -0.130000      803,690   
Digiter, Appliences 7                9044890      0.057181      380,139   
                    8                4119730     -0.540000      341,060   
                    9                2404690     -0.410000      300,333   
Fashion_Accessories 7               15256860      0.052881      226,420   
                    8                7628340     -0.500000      209,674   
                    9               14844660      0.940000      240,026   
Fashion_Clothes     7               11384490      0.041898      380,096   
                    8                9390710     -0.170000      311,563   
                    9               23468480      1.490000      454,911   
Foods               7               29080835      0.074095      459,618   
                    8               25047402     -0.130000      517,049   
                    9               24010572     -0.040000      519,648   
Interier            7                3166590     -0.011171       68,424   
                    8                1467600     -0.530000       61,196   
                    9                1510980      0.020000       64,340   
Living, Health      7                4180500      0.140639      302,097   
                    8                3410860     -0.180000      308,283   
                    9                4109010      0.200000      305,203   
Others              7                 362100      0.118120       52,205   
                    8                1604330      3.430000       51,134   
                    9                1397540     -0.120000       50,228   
Sports, Leisure     7                 671210     -0.031229       97,407   
                    8                 219700     -0.670000      113,798   
                    9                 277630      0.260000       72,839   
Travel, Culture     7                 969300      0.074720      813,189   
                    8                 390263     -0.590000      806,915   
                    9                 824670      1.110000      713,819   

                                    trend_korea  trend_mean  
clac_nm0            trans_tm_month                           
Baby                7                  0.067508    0.067508  
                    8                  0.051716    0.285858  
                    9                 -0.000695   -0.085348  
Beauty              7                  0.050069    0.050069  
                    8                  0.063380    0.021690  
                    9                  0.085057   -0.022471  
Digiter, Appliences 7                  0.057181    0.057181  
                    8                 -0.102802   -0.321401  
                    9                 -0.119413   -0.264707  
Fashion_Accessories 7                  0.052881    0.052881  
                    8                 -0.073960   -0.286980  
                    9                  0.144758    0.542379  
Fashion_Clothes     7                  0.041898    0.041898  
                    8                 -0.180304   -0.175152  
                    9                  0.460093    0.975047  
Foods               7                  0.074095    0.074095  
                    8                  0.124954   -0.002523  
                    9                  0.005027   -0.017487  
Interier            7                 -0.011171   -0.011171  
                    8                 -0.105635   -0.317818  
                    9                  0.051376 

In [156]:
recomend_20=transaction_20_payment.pivot_table(index ='trans_tm_month', values = 'trend_mean', aggfunc='max')
recomend_30=transaction_30_payment.pivot_table(index ='trans_tm_month', values = 'trend_mean', aggfunc='max')
recomend_40=transaction_40_payment.pivot_table(index ='trans_tm_month', values = 'trend_mean', aggfunc='max')
recomend_50=transaction_50_payment.pivot_table(index ='trans_tm_month', values = 'trend_mean', aggfunc='max')
recomend_60=transaction_60_payment.pivot_table(index ='trans_tm_month', values = 'trend_mean', aggfunc='max')

In [168]:
recomend_20_final=recomend_20.merge(transaction_20_payment).drop(['buy_am', 'trend_lpoint', 'buy_am_korea', 'trend_korea'], axis=1).set_index('trans_tm_month')
recomend_30_final=recomend_30.merge(transaction_30_payment).drop(['buy_am', 'trend_lpoint', 'buy_am_korea', 'trend_korea'], axis=1).set_index('trans_tm_month')
recomend_40_final=recomend_40.merge(transaction_40_payment).drop(['buy_am', 'trend_lpoint', 'buy_am_korea', 'trend_korea'], axis=1).set_index('trans_tm_month')
recomend_50_final=recomend_50.merge(transaction_50_payment).drop(['buy_am', 'trend_lpoint', 'buy_am_korea', 'trend_korea'], axis=1).set_index('trans_tm_month')
recomend_60_final=recomend_60.merge(transaction_60_payment).drop(['buy_am', 'trend_lpoint', 'buy_am_korea', 'trend_korea'], axis=1).set_index('trans_tm_month')

In [169]:
recomend_20_final

,trend_mean,clac_nm0
trans_tm_month,,
7,0.140639,"Living, Health"
8,1.704742,Others
9,0.975047,Fashion_Clothes
